In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Project/2023/BIGCON/데이터_합치는_코드

/content/drive/MyDrive/Project/2023/BIGCON/데이터_합치는_코드


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import geopandas as gpd

In [5]:
seoul_pnu = gpd.read_file('서울시_필지_LX맵/서울시_필지전체.shp', encoding = 'cp949')
seoul_pnu.head(2)

,PNU,SGG_CD,SGG_NM,EMD_CD,EMD_NM,geometry
0,1111010100100010000,11110,종로구,11110101,청운동,"POLYGON ((197211.824 554668.412, 197240.351 55..."
1,1111010100100010001,11110,종로구,11110101,청운동,"POLYGON ((197300.251 554580.847, 197298.254 55..."


In [7]:
raw_df = pd.read_csv('필지단위 소상공인 매출등급 정보.csv', encoding = 'cp949')
test = pd.read_excel('소상공인 매출등급 예상 대상 필지.xlsx')
raw_df.head(2)

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD
0,202201,1111010100100030100,11110101,A03,4
1,202201,1111010100100480000,11110101,A01,2


### raw_data 만들기(최종 데이터셋 만들때 기준이 될 데이터)

In [8]:
# train이면 1, test면 0 변수 생성
raw_df['train'] = 1
# test 데이터는 202301과 202302가 열별로 되어있어 각각 데이터프레임 만든 후 raw데이터 밑으로 붙이기
test_2301 = test[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2302 = test[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2301['DATA_CRTR_YM'] = 202301
test_2302['DATA_CRTR_YM'] = 202302
test_2301['train'] = 0
test_2302['train'] = 0

<ipython-input-8-d6a3307fbb4c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2301['DATA_CRTR_YM'] = 202301
<ipython-input-8-d6a3307fbb4c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2302['DATA_CRTR_YM'] = 202302


In [9]:
test_2301 = test_2301[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'train']]
test_2302 = test_2302[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'train']]

In [10]:
raw_data = pd.concat([raw_df, test_2301, test_2302])

In [11]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176040 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1176040 non-null  int64  
 1   LT_UNQ_NO     1176040 non-null  int64  
 2   STDG_EMD_CD   1176040 non-null  int64  
 3   INDUSTRY_CD   1176040 non-null  object 
 4   SLS_GRD       1156040 non-null  float64
 5   train         1176040 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 62.8+ MB


### 필지번호와 지하철역 이름 매칭

#### 서울시 필지 shp 중에서 raw_data에 존재하는 필지만 뽑은 후, geometry 붙여놓기

In [12]:
temp = raw_data.drop_duplicates(subset = 'LT_UNQ_NO', keep = 'first', ignore_index = True)
temp = temp[['LT_UNQ_NO']]
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74303 entries, 0 to 74302
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   LT_UNQ_NO  74303 non-null  int64
dtypes: int64(1)
memory usage: 580.6 KB


In [15]:
seoul_pnu = seoul_pnu.astype({'PNU' : 'int'})

In [28]:
pnu_data = pd.merge(left = seoul_pnu, right = temp, left_on = 'PNU', right_on = 'LT_UNQ_NO', how = 'right')
pnu_data.head(2)

,PNU,SGG_CD,SGG_NM,EMD_CD,EMD_NM,geometry,LT_UNQ_NO
0,1.111010e+18,11110,종로구,11110101,청운동,"POLYGON ((197073.176 554737.046, 197092.636 55...",1111010100100030100
1,1.111010e+18,11110,종로구,11110101,청운동,"POLYGON ((197305.633 554178.725, 197306.531 55...",1111010100100480000


In [29]:
pnu_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 74303 entries, 0 to 74302
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   PNU        74199 non-null  float64 
 1   SGG_CD     74199 non-null  object  
 2   SGG_NM     74199 non-null  object  
 3   EMD_CD     74199 non-null  object  
 4   EMD_NM     74199 non-null  object  
 5   geometry   74199 non-null  geometry
 6   LT_UNQ_NO  74303 non-null  int64   
dtypes: float64(1), geometry(1), int64(1), object(4)
memory usage: 4.5+ MB


In [30]:
pnu_data.drop(['PNU'], axis = 1, inplace = True)
pnu_data = pnu_data[['LT_UNQ_NO', 'SGG_CD', 'SGG_NM', 'EMD_CD', 'EMD_NM', 'geometry']]
pnu_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 74303 entries, 0 to 74302
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   LT_UNQ_NO  74303 non-null  int64   
 1   SGG_CD     74199 non-null  object  
 2   SGG_NM     74199 non-null  object  
 3   EMD_CD     74199 non-null  object  
 4   EMD_NM     74199 non-null  object  
 5   geometry   74199 non-null  geometry
dtypes: geometry(1), int64(1), object(4)
memory usage: 4.0+ MB


#### pnu_data에 역 이름 붙여놓기

In [19]:
subway = gpd.read_file('수도권 지하철 공간데이터/2019/TB_O_SB_STATN_2019.shp', encoding = 'cp949')
subway.head(2)

,ID,STN_IDN,TGRP_CODE,STN_NM,BB_RT,US_YY_DD,TM_X,TM_Y,geometry
0,103.0,416,지하철,미아사거리,4,201911,202653.1120,557083.2120,POINT (202653.112 557083.212)
1,104.0,417,지하철,길음,4,201911,202211.9992,555986.0728,POINT (202211.999 555986.073)


In [20]:
subway.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ID         680 non-null    float64 
 1   STN_IDN    680 non-null    object  
 2   TGRP_CODE  680 non-null    object  
 3   STN_NM     680 non-null    object  
 4   BB_RT      680 non-null    object  
 5   US_YY_DD   680 non-null    object  
 6   TM_X       680 non-null    float64 
 7   TM_Y       680 non-null    float64 
 8   geometry   680 non-null    geometry
dtypes: float64(3), geometry(1), object(5)
memory usage: 47.9+ KB


In [21]:
import re

In [22]:
pattern = r'\([^)]*\)'

In [23]:
subway['STN_NM'] = subway['STN_NM'].apply(lambda x: re.sub(pattern, repl='', string = x))

In [24]:
subway = subway.to_crs(epsg = 5186)

In [32]:
pnu_data = gpd.sjoin(left_df = pnu_data, right_df = subway, how = 'left', predicate = 'intersects')
pnu_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 74331 entries, 0 to 74302
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   LT_UNQ_NO    74331 non-null  int64   
 1   SGG_CD       74227 non-null  object  
 2   SGG_NM       74227 non-null  object  
 3   EMD_CD       74227 non-null  object  
 4   EMD_NM       74227 non-null  object  
 5   geometry     74227 non-null  geometry
 6   index_right  100 non-null    float64 
 7   ID           100 non-null    float64 
 8   STN_IDN      100 non-null    object  
 9   TGRP_CODE    100 non-null    object  
 10  STN_NM       100 non-null    object  
 11  BB_RT        100 non-null    object  
 12  US_YY_DD     100 non-null    object  
 13  TM_X         100 non-null    float64 
 14  TM_Y         100 non-null    float64 
dtypes: float64(4), geometry(1), int64(1), object(9)
memory usage: 9.1+ MB


In [34]:
pnu_data.head(2)

,LT_UNQ_NO,SGG_CD,SGG_NM,EMD_CD,EMD_NM,geometry,index_right,ID,STN_IDN,TGRP_CODE,STN_NM,BB_RT,US_YY_DD,TM_X,TM_Y
0,1111010100100030100,11110,종로구,11110101,청운동,"POLYGON ((197073.176 554737.046, 197092.636 55...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1111010100100480000,11110,종로구,11110101,청운동,"POLYGON ((197305.633 554178.725, 197306.531 55...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
pnu_data = pnu_data[['LT_UNQ_NO', 'geometry', 'STN_NM']]
pnu_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 74331 entries, 0 to 74302
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   LT_UNQ_NO  74331 non-null  int64   
 1   geometry   74227 non-null  geometry
 2   STN_NM     100 non-null    object  
dtypes: geometry(1), int64(1), object(1)
memory usage: 2.3+ MB


### raw_data에 역별 승하차 인원수 붙이기

In [37]:
subway_pop = pd.read_csv('지하철역별 데이터(9호선포함).csv', encoding = 'cp949')
subway_pop.head(2)

,수송일자,역명,출근시간_승차수,출근시간_하차수,주말_하차수
0,202201,가락시장,50376.0,86354.0,83622.0
1,202201,가산디지털단지,37396.0,551963.0,104379.0


In [38]:
subway_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3512 entries, 0 to 3511
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   수송일자      3512 non-null   int64  
 1   역명        3512 non-null   object 
 2   출근시간_승차수  3512 non-null   float64
 3   출근시간_하차수  3512 non-null   float64
 4   주말_하차수    3512 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 137.3+ KB


In [39]:
subway_pop['역명'] = subway_pop['역명'].apply(lambda x: re.sub(pattern, repl='', string = x))

#### raw_data에 pnu_data 이용해서 역 이름 붙이기

In [40]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176040 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1176040 non-null  int64  
 1   LT_UNQ_NO     1176040 non-null  int64  
 2   STDG_EMD_CD   1176040 non-null  int64  
 3   INDUSTRY_CD   1176040 non-null  object 
 4   SLS_GRD       1156040 non-null  float64
 5   train         1176040 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 62.8+ MB


In [41]:
pnu_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 74331 entries, 0 to 74302
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   LT_UNQ_NO  74331 non-null  int64   
 1   geometry   74227 non-null  geometry
 2   STN_NM     100 non-null    object  
dtypes: geometry(1), int64(1), object(1)
memory usage: 2.3+ MB


In [43]:
raw_data2 = pd.merge(left = raw_data, right = pnu_data, on = 'LT_UNQ_NO', how = 'left')

In [44]:
raw_data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1177341 entries, 0 to 1177340
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype   
---  ------        --------------    -----   
 0   DATA_CRTR_YM  1177341 non-null  int64   
 1   LT_UNQ_NO     1177341 non-null  int64   
 2   STDG_EMD_CD   1177341 non-null  int64   
 3   INDUSTRY_CD   1177341 non-null  object  
 4   SLS_GRD       1157315 non-null  float64 
 5   train         1177341 non-null  int64   
 6   geometry      1174203 non-null  geometry
 7   STN_NM        3155 non-null     object  
dtypes: float64(1), geometry(1), int64(4), object(2)
memory usage: 80.8+ MB


#### raw_data2에 날짜 맞춰서 승하차수 붙이기

In [46]:
raw_data2[raw_data2['DATA_CRTR_YM'] == 202201]

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train,geometry,STN_NM
0,202201,1111010100100030100,11110101,A03,4.0,1,"POLYGON ((197073.176 554737.046, 197092.636 55...",NaN
1,202201,1111010100100480000,11110101,A01,2.0,1,"POLYGON ((197305.633 554178.725, 197306.531 55...",NaN
2,202201,1111010100100590001,11110101,A11,3.0,1,"POLYGON ((197332.536 554075.806, 197333.169 55...",NaN
3,202201,1111010100100590005,11110101,A03,2.0,1,"POLYGON ((197292.491 554081.166, 197302.207 55...",NaN
4,202201,1111010100100690000,11110101,A01,2.0,1,"POLYGON ((197368.683 554110.560, 197369.976 55...",NaN
...,...,...,...,...,...,...,...,...
93996,202201,1174011000106840000,11740110,A02,5.0,1,"POLYGON ((215698.063 551466.673, 215692.889 55...",NaN
93997,202201,1174011000106950000,11740110,A04,3.0,1,"POLYGON ((215876.077 551146.763, 215876.628 55...",NaN
93998,202201,1174011000106950000,11740110,A03,3.0,1,"POLYGON ((215876.077 551146.763, 215876.628 55...",NaN
93999,202201,1174011000106990000,11740110,A01,4.0,1,"POLYGON ((215880.621 551018.995, 215686.352 55...",NaN


In [49]:
s2201 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202201], right = subway_pop[subway_pop['수송일자'] == 202201], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2202 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202202], right = subway_pop[subway_pop['수송일자'] == 202202], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2203 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202203], right = subway_pop[subway_pop['수송일자'] == 202203], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2204 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202204], right = subway_pop[subway_pop['수송일자'] == 202204], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2205 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202205], right = subway_pop[subway_pop['수송일자'] == 202205], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2206 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202206], right = subway_pop[subway_pop['수송일자'] == 202206], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2207 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202207], right = subway_pop[subway_pop['수송일자'] == 202207], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2208 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202208], right = subway_pop[subway_pop['수송일자'] == 202208], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2209 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202209], right = subway_pop[subway_pop['수송일자'] == 202209], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2210 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202210], right = subway_pop[subway_pop['수송일자'] == 202210], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2211 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202211], right = subway_pop[subway_pop['수송일자'] == 202211], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2212 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202212], right = subway_pop[subway_pop['수송일자'] == 202212], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2301 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202301], right = subway_pop[subway_pop['수송일자'] == 202301], left_on = 'STN_NM', right_on = '역명', how = 'left')
s2302 = pd.merge(left = raw_data2[raw_data2['DATA_CRTR_YM'] == 202302], right = subway_pop[subway_pop['수송일자'] == 202302], left_on = 'STN_NM', right_on = '역명', how = 'left')

In [51]:
raw_data3 = pd.concat([s2201, s2202, s2203, s2204, s2205, s2206, s2207, s2208, s2209, s2210, s2211, s2212, s2301, s2302])

In [52]:
raw_data3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1177341 entries, 0 to 10012
Data columns (total 13 columns):
 #   Column        Non-Null Count    Dtype   
---  ------        --------------    -----   
 0   DATA_CRTR_YM  1177341 non-null  int64   
 1   LT_UNQ_NO     1177341 non-null  int64   
 2   STDG_EMD_CD   1177341 non-null  int64   
 3   INDUSTRY_CD   1177341 non-null  object  
 4   SLS_GRD       1157315 non-null  float64 
 5   train         1177341 non-null  int64   
 6   geometry      1174203 non-null  geometry
 7   STN_NM        3155 non-null     object  
 8   수송일자          2610 non-null     float64 
 9   역명            2610 non-null     object  
 10  출근시간_승차수      2610 non-null     float64 
 11  출근시간_하차수      2610 non-null     float64 
 12  주말_하차수        2610 non-null     float64 
dtypes: float64(5), geometry(1), int64(4), object(3)
memory usage: 125.8+ MB


In [53]:
raw_data3.drop(['geometry', 'STN_NM', '수송일자', '역명'], axis = 1, inplace = True)

In [61]:
# NaN 0으로 대체
raw_data3.loc[raw_data3['출근시간_승차수'] != raw_data3['출근시간_승차수'], '출근시간_승차수'] = 0
raw_data3.loc[raw_data3['출근시간_하차수'] != raw_data3['출근시간_하차수'], '출근시간_하차수'] = 0
raw_data3.loc[raw_data3['주말_하차수'] != raw_data3['주말_하차수'], '주말_하차수'] = 0

In [62]:
raw_data3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1177341 entries, 0 to 10012
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1177341 non-null  int64  
 1   LT_UNQ_NO     1177341 non-null  int64  
 2   STDG_EMD_CD   1177341 non-null  int64  
 3   INDUSTRY_CD   1177341 non-null  object 
 4   SLS_GRD       1157315 non-null  float64
 5   train         1177341 non-null  int64  
 6   출근시간_승차수      1177341 non-null  float64
 7   출근시간_하차수      1177341 non-null  float64
 8   주말_하차수        1177341 non-null  float64
dtypes: float64(4), int64(4), object(1)
memory usage: 89.8+ MB


In [66]:
# 한 필지에 여러 개의 역이 있는 경우, 승하차수 다 합치기
raw_data4 = raw_data3.groupby(['DATA_CRTR_YM', 'LT_UNQ_NO'])[['출근시간_승차수', '출근시간_하차수', '주말_하차수']].sum().reset_index()

In [68]:
# raw_data랑 merge
raw_data4 = pd.merge(left = raw_data, right = raw_data4, how = 'left', on = ['DATA_CRTR_YM', 'LT_UNQ_NO'])
raw_data4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176040 entries, 0 to 1176039
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1176040 non-null  int64  
 1   LT_UNQ_NO     1176040 non-null  int64  
 2   STDG_EMD_CD   1176040 non-null  int64  
 3   INDUSTRY_CD   1176040 non-null  object 
 4   SLS_GRD       1156040 non-null  float64
 5   train         1176040 non-null  int64  
 6   출근시간_승차수      1176040 non-null  float64
 7   출근시간_하차수      1176040 non-null  float64
 8   주말_하차수        1176040 non-null  float64
dtypes: float64(4), int64(4), object(1)
memory usage: 89.7+ MB


### 기존 raw_data(데이터 분석 시 사용하던거)랑 합치기

In [69]:
data = pd.read_csv('/content/drive/MyDrive/Project/2023/BIGCON/raw_data.csv', encoding = 'cp949')
data.head(2)

,DATA_CRTR_YM,LT_UNQ_NO,STDG_EMD_CD,INDUSTRY_CD,SLS_GRD,train,전체점포수,프랜차이즈점포수,일반점포수,길단위유동인구,...,생활물가지수,부동산거래대비유동인구,공실률대비매매가임대료,젠트리피케이션,지하철개수,스타벅스개수,대학교여부,65세이상,65세이상_남,65세이상_여
0,202201,1111010100100030100,11110101,A03,4.0,1,85.84,1.45,84.39,14929.0,...,104.55,19.76,73.540769,51.190526,0.0,0.0,0,2144.0,875.0,1269.0
1,202201,1156011100104460000,11560111,A01,5.0,1,295.00,23.00,272.00,92745.0,...,104.55,51.69,68.320000,43.180000,2.0,0.0,0,3386.0,1506.0,1880.0


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176040 entries, 0 to 1176039
Data columns (total 29 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1176040 non-null  int64  
 1   LT_UNQ_NO     1176040 non-null  int64  
 2   STDG_EMD_CD   1176040 non-null  int64  
 3   INDUSTRY_CD   1176040 non-null  object 
 4   SLS_GRD       1156040 non-null  float64
 5   train         1176040 non-null  int64  
 6   전체점포수         1176040 non-null  float64
 7   프랜차이즈점포수      1176040 non-null  float64
 8   일반점포수         1176040 non-null  float64
 9   길단위유동인구       1176040 non-null  float64
 10  주거인구          1176040 non-null  float64
 11  직장인구          1176040 non-null  float64
 12  개업수           1176040 non-null  float64
 13  폐업수           1176040 non-null  float64
 14  개업률           1176040 non-null  float64
 15  폐업률           1176040 non-null  float64
 16  전체 임대료        1176040 non-null  float64
 17  1층 임대료        1176040 non-n

In [77]:
merge_df = pd.merge(left = data, right = raw_data4, how = 'left', on = ['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'SLS_GRD', 'train'])
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1176040 entries, 0 to 1176039
Data columns (total 32 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1176040 non-null  int64  
 1   LT_UNQ_NO     1176040 non-null  int64  
 2   STDG_EMD_CD   1176040 non-null  int64  
 3   INDUSTRY_CD   1176040 non-null  object 
 4   SLS_GRD       1156040 non-null  float64
 5   train         1176040 non-null  int64  
 6   전체점포수         1176040 non-null  float64
 7   프랜차이즈점포수      1176040 non-null  float64
 8   일반점포수         1176040 non-null  float64
 9   길단위유동인구       1176040 non-null  float64
 10  주거인구          1176040 non-null  float64
 11  직장인구          1176040 non-null  float64
 12  개업수           1176040 non-null  float64
 13  폐업수           1176040 non-null  float64
 14  개업률           1176040 non-null  float64
 15  폐업률           1176040 non-null  float64
 16  전체 임대료        1176040 non-null  float64
 17  1층 임대료        1176040 non-n

In [78]:
merge_df.columns

Index(['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD', 'SLS_GRD',
       'train', '전체점포수', '프랜차이즈점포수', '일반점포수', '길단위유동인구', '주거인구', '직장인구', '개업수',
       '폐업수', '개업률', '폐업률', '전체 임대료', '1층 임대료', '1층 외 임대료', '생활물가지수',
       '부동산거래대비유동인구', '공실률대비매매가임대료', '젠트리피케이션', '지하철개수', '스타벅스개수', '대학교여부',
       '65세이상', '65세이상_남', '65세이상_여', '출근시간_승차수', '출근시간_하차수', '주말_하차수'],
      dtype='object')

In [82]:
merge_df.to_csv('raw_data_final.csv', index = False, encoding = 'cp949')